In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCSn90DAxCF3yRs6mFK0-8TA/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 109


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

23:52
14:38
27:39
26:28
30:42
24:54
29:52
25:24
24:18
20:29
21:51
21:39
23:05
31:02
25:52
23:20
25:44
33:37
24:17
28:34
29:44
34:32
22:10
22:17
24:40
25:57
12:35
28:23
21:42
24:18
27:57
19:11
14:49
20:48
24:00
20:45
21:49
25:28
23:31
25:44
28:23
26:40
23:21
27:56
25:09
26:44
21:19
15:10
23:54
21:53
19:08
22:19
30:12
21:08
22:59
19:56
22:24
22:41
21:06
17:41
23:09
22:35
16:06
22:12
22:13
24:35
22:43
24:11
21:14
17:39
6:37
19:28
16:05
21:06
22:52
15:01
23:29
25:02
17:52
22:06
20:17
20:10
21:25
16:28
18:39
20:37
20:23
18:07
29:59
22:42
17:02
15:41
16:22
15:50
16:48
14:12
13:08
13:39
14:45
12:22
8:15
8:12
11:39
13:02
11:36
4:36
6:42
7:26
4:10


In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [5]:
for i in range(len(video_list2)):
    #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
    thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
    title = video_list2[i].find('a',{'id':'video-title'}).text # 2 
    
    # 여기서부터 영상 클릭해서 들어간 페이지
    click_url = park_url[i]
    browser.get(click_url)
    time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
    # body = browser.find_element_by_tag_name('body')
    browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
    time.sleep(3)
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
   
    # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
    category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
    # ytd-metadata-row-renderer.style-scope a
    
    view_count = html.find('span', {'class': 'view-count'}).text # 4
    
    # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
    # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
    likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
    'aria-label':re.compile('좋아요')}).text+'개'
    dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
    'aria-label':re.compile('싫어요')}).text+'개'
    upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
    
    print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    thumbnail_list.append(thumbnail)
    title_list.append(title)
    category_list.append(category)
    view_count_list.append(view_count)
    likes_count_list.append(likes_count)
    dislikes_count_list.append(dislikes_count)
    upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/f80KUtt-zcc/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAYzqGrm-aM88vIgDBQt24VNPc2hw 제목: ENG)출근일상,친구들이랑 비빔밥 해 먹고 라탄 원데이클래스 듣는 뿌듯한 일상 그리고 생일선물 풀어보기 카테고리 Yêu Chim Chim 조회수 조회수 498,730회 좋아요수 1.5만개 싫어요수 181개 게시일 2019. 12. 25.


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
print(len(video_length_list),len(thumbnail_list))

In [ ]:
import pandas as pd

insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    'video_length' : video_length_list
                    }) 
insert_data

In [ ]:
insert_data.to_csv("ondo.csv",encoding="utf8")

In [ ]:
# 소련여자, 박막례, 최고다윽박, ondo온도, 오킹, 오늘의 하늘,튜브몬,박병창